In [1]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [2]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 Financials Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [3]:
# Get close data for S&P 500 Information Technology Index
market_index_data = Utils.get_historical_data("^SP500-40", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  30.755467685481896
Market Index Expected daily Return in %:  0.11144401773318496
Market Index daily Variance in %:  0.006544669226452062


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [4]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Financial Sector ETFs

We will now download the data for US Financial Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets excluding leveraged and index ETF:
1. Financial Select Sector SPDR Fund (XLF)
2. Vanguard Financials ETF (VFH)
3. iShares US Financials ETF (IYF)
4. SPDR S&P Regional Banking ETF (KRE)
5. Invesco KBW Bank ETF (KBWB)

Source: https://etfdb.com/etfdb-category/technology-equities/ 

In [27]:
# get the data
xlf_data = Utils.get_historical_data("XLF", start_date, end_date)
vfh_data = Utils.get_historical_data("VFH", start_date, end_date)
iyf_data = Utils.get_historical_data("IYF", start_date, end_date)
kre_data = Utils.get_historical_data("KRE", start_date, end_date)
kbwb_data = Utils.get_historical_data("KBWB", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [28]:
# check for any missing data
print(market_index_data.isnull().sum())
print(xlf_data.isnull().sum())
print(vfh_data.isnull().sum())
print(iyf_data.isnull().sum())
print(kre_data.isnull().sum())
print(kbwb_data.isnull().sum())

Adj Close        0
simple_return    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


In [29]:

xlf_simple_return = Utils.calculate_simple_return(xlf_data)
vfh_simple_return = Utils.calculate_simple_return(vfh_data)
iyf_simple_return = Utils.calculate_simple_return(iyf_data)
kre_simple_return = Utils.calculate_simple_return(kre_data)
kbwb_simple_return = Utils.calculate_simple_return(kbwb_data)


simple_returns_dict = {
    "XLF": xlf_simple_return,
    "VFH": vfh_simple_return,
    "IYF": iyf_simple_return,
    "KRE": kre_simple_return,
    "KBWB": kbwb_simple_return
}

for ticker, simple_return in simple_returns_dict.items():
    print(f"{ticker} -> Simple Return from {start_date} - {end_date} in %: ", simple_return * 100)

XLF -> Simple Return from 2023-04-01 - 2024-04-01 in %:  33.09317548673516
VFH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  34.56980207188251
IYF -> Simple Return from 2023-04-01 - 2024-04-01 in %:  36.68568980169879
KRE -> Simple Return from 2023-04-01 - 2024-04-01 in %:  19.8646954702302
KBWB -> Simple Return from 2023-04-01 - 2024-04-01 in %:  33.28175448556869


In [30]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for Financials ETFs and Market Index")
fig.add_scatter(x=xlf_data.index, y=xlf_data['simple_return'], mode='lines', name='VGT')
fig.add_scatter(x=vfh_data.index, y=vfh_data['simple_return'], mode='lines', name='VHT')
fig.add_scatter(x=iyf_data.index, y=iyf_data['simple_return'], mode='lines', name='IYF')
fig.add_scatter(x=kre_data.index, y=kre_data['simple_return'], mode='lines', name='KRE')
fig.add_scatter(x=kbwb_data.index, y=kbwb_data['simple_return'], mode='lines', name='KBWB')

fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that apart from SPDR S&P 500 Regional Banking ETF (KRE), all Financial ETFs had a similar increasing trend narrowly beating the market index with iShares US Financial ETFs (IYF) registering the highest return. This could be due to fact that these ETFs hold a large nummber of shares in large-cap financial companies which did quite well in the last year

In [31]:
xlf_daily_returns = Utils.calculate_daily_returns(xlf_data)
vfh_daily_returns = Utils.calculate_daily_returns(vfh_data)
iyf_daily_returns = Utils.calculate_daily_returns(iyf_data)
kre_daily_returns = Utils.calculate_daily_returns(kre_data)
kbwb_daily_returns = Utils.calculate_daily_returns(kbwb_data)

daily_returns_df = pd.DataFrame([xlf_daily_returns, vfh_daily_returns, iyf_daily_returns, kre_daily_returns, kbwb_daily_returns]).T
daily_returns_df.columns = ["XLF", "VFH", "IYF", "KRE", "KBWB"]

In [33]:
# we can now plot the daily returns for all the stocks in sub plots
fig = make_subplots(rows=3, cols=2, subplot_titles=("XLF", "VFH", "IYF", "KRE", "KBWB", "Market Index"))

fig.add_scatter(x=xlf_data.index, y=xlf_daily_returns, mode='lines', name='VGT', row=1, col=1)
fig.add_scatter(x=vfh_data.index, y=vfh_daily_returns, mode='lines', name='VHT', row=1, col=2)
fig.add_scatter(x=iyf_data.index, y=iyf_daily_returns, mode='lines', name='IYF', row=2, col=1)
fig.add_scatter(x=kre_data.index, y=kre_daily_returns, mode='lines', name='KRE', row=2, col=2)
fig.add_scatter(x=kbwb_data.index, y=kbwb_daily_returns, mode='lines', name='FAS', row=3, col=1)
fig.add_scatter(x=market_index_data.index, y=market_index_daily_returns, mode='lines', name='Market Index', row=3, col=2)

fig.update_layout(title_text="Daily Returns for Financial ETFs and Market Index", height=1000, width=1000)
fig.show()


In [34]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,XLF,VFH,IYF,KRE,KBWB
XLF,0.000065,0.000070,0.000070,0.000129,0.000106
VFH,0.000070,0.000078,0.000077,0.000156,0.000123
IYF,0.000070,0.000077,0.000078,0.000153,0.000123
KRE,0.000129,0.000156,0.000153,0.000449,0.000300
KBWB,0.000106,0.000123,0.000123,0.000300,0.000231


In [35]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,XLF,VFH,IYF,KRE,KBWB
XLF,1.000000,0.982463,0.975540,0.750158,0.862261
VFH,0.982463,1.000000,0.986827,0.829206,0.911705
IYF,0.975540,0.986827,1.000000,0.820302,0.918613
KRE,0.750158,0.829206,0.820302,1.000000,0.929446
KBWB,0.862261,0.911705,0.918613,0.929446,1.000000


Similar to Technology ETFs, Financial ETFs have a high correlation to each other as well. This is expected as they have similar total returns.

In [36]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

XLF     0.008089
VFH     0.008851
IYF     0.008816
KRE     0.021191
KBWB    0.015210
dtype: float64

In [37]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

XLF     0.001186
VFH     0.001237
IYF     0.001300
KRE     0.000953
KBWB    0.001274
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [38]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [39]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [41]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "XLF": markowitz_weights_with_ss[0],
    "VFH": markowitz_weights_with_ss[1],
    "IYF": markowitz_weights_with_ss[2],
    "KRE": markowitz_weights_with_ss[3],
    "KBWB": markowitz_weights_with_ss[4]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight * 100)

XLF -> Markowitz Weights with Short Selling in %:  -276.02792840104564
VFH -> Markowitz Weights with Short Selling in %:  170.17500219939527
IYF -> Markowitz Weights with Short Selling in %:  296.2880955821801
KRE -> Markowitz Weights with Short Selling in %:  -48.63228349928861
KBWB -> Markowitz Weights with Short Selling in %:  -41.80288588124116


In [42]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [44]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "XLF": markowitz_weights_without_ss[0],
    "VFH": markowitz_weights_without_ss[1],
    "IYF": markowitz_weights_without_ss[2],
    "KRE": markowitz_weights_without_ss[3],
    "KBWB": markowitz_weights_without_ss[4]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

XLF -> Markowitz Weights without Short Selling in %:  0.0
VFH -> Markowitz Weights without Short Selling in %:  0.0
IYF -> Markowitz Weights without Short Selling in %:  99.9999999996491
KRE -> Markowitz Weights without Short Selling in %:  8.843786813983456e-10
KBWB -> Markowitz Weights without Short Selling in %:  1.4443446438860974e-10


In [45]:
volatility_with_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_with_ss, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)

Volatility with Short Selling in %:  0.8070348595047843


In [46]:
portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  2.6765584576206773


In [47]:
# calculate sharpe ratio
sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Sharpe Ratio with Short Selling:  0.3593164661625195


In [48]:
volatility_without_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)

Volatility without Short Selling in %:  0.881585477309298


In [49]:
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  2.0630701925814847


In [50]:
# calculate sharpe ratio
sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Sharpe Ratio without Short Selling:  0.2270775897905066


From the results above, we observe that there is only a difference 0.6% in mean portfolio returns when short selling is not allowed. 

We also observe that without weights constraint in optimization without short selling, IYF account for around 99% proportion in the portfolio. To promote diversification , we will add a constraint that allows a maximum of 30% in each ETF

In [51]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.3)

In [52]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "XLF": markowitz_weights_without_ss_pct_constraint[0],
    "VFH": markowitz_weights_without_ss_pct_constraint[1],
    "IYF": markowitz_weights_without_ss_pct_constraint[2],
    "KRE": markowitz_weights_without_ss_pct_constraint[3],
    "KBWB": markowitz_weights_without_ss_pct_constraint[4]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

XLF -> Markowitz Weights without Short Selling and 30% constraint in %:  29.99999999982742
VFH -> Markowitz Weights without Short Selling and 30% constraint in %:  29.999999999817952
IYF -> Markowitz Weights without Short Selling and 30% constraint in %:  29.999999999798852
KRE -> Markowitz Weights without Short Selling and 30% constraint in %:  4.805461584211912e-10
KBWB -> Markowitz Weights without Short Selling and 30% constraint in %:  10.000000000075232


In [53]:
volatility_without_ss_with_constraint = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss_pct_constraint, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)

Volatility with Short Selling and Constraint in %:  0.9078276238381376


In [54]:
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

Mean Portfolio Return with Short Selling and Constraint in %:  1.9750048465967753


In [55]:
# find the Sharpe Ratio
sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Sharpe Ratio with Short Selling and Constraint:  0.2051080547136239


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Healthcare Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [56]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [58]:
# run SIMM for params
xlf_alpha, xlf_beta, xlf_tau = get_simm_params(xlf_daily_returns, market_index_daily_returns)
vfh_alpha, vfh_beta, vfh_tau = get_simm_params(vfh_daily_returns, market_index_daily_returns)
iyf_alpha, iyf_beta, iyf_tau = get_simm_params(iyf_daily_returns, market_index_daily_returns)
kre_alpha, kre_beta, kre_tau = get_simm_params(kre_daily_returns, market_index_daily_returns)
kbwb_alpha, kbwb_beta, kbwb_tau = get_simm_params(kbwb_daily_returns, market_index_daily_returns)

In [59]:
# get expected returns using alpha, beta and expected market return
xlf_expected_return = xlf_alpha + (xlf_beta * market_index_data_expected_return)
vfh_expected_return = vfh_alpha + (vfh_beta * market_index_data_expected_return)
iyf_expected_return = iyf_alpha + (iyf_beta * market_index_data_expected_return)
kre_expected_return = kre_alpha + (kre_beta * market_index_data_expected_return)
kbwb_expected_return = kbwb_alpha + (kbwb_beta * market_index_data_expected_return)

In [60]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "XLF": xlf_expected_return,
    "VFH": vfh_expected_return,
    "IYF": iyf_expected_return,
    "KRE": kre_expected_return,
    "KBWB": kbwb_expected_return
}

alpha_dict = {
    "XLF": xlf_alpha,
    "VFH": vfh_alpha,
    "IYF": iyf_alpha,
    "KRE": kre_alpha,
    "KBWB": kbwb_alpha
}

beta_dict = {
    "XLF": xlf_beta,
    "VFH": vfh_beta,
    "IYF": iyf_beta,
    "KRE": kre_beta,
    "KBWB": kbwb_beta
}

tau_dict = {
    "XLF": xlf_tau,
    "VFH": vfh_tau,
    "IYF": iyf_tau,
    "KRE": kre_tau,
    "KBWB": kbwb_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

XLF -> Expected Daily Return in %:  0.11859703454364787
XLF -> Alpha:  7.463044597595779e-05
XLF -> Beta:  0.9972180849772023
XLF -> Tau:  0.0005986329752819318

VFH -> Expected Daily Return in %:  0.12369355392750274
VFH -> Alpha:  3.563378567958037e-05
VFH -> Beta:  1.0779418922885193
VFH -> Tau:  0.001519131564152786

IYF -> Expected Daily Return in %:  0.12996120635278027
IYF -> Alpha:  0.00011145099402801081
IYF -> Beta:  1.066150605180479
IYF -> Tau:  0.0018278579654365068

KRE -> Expected Daily Return in %:  0.09528238578853766
KRE -> Alpha:  -0.0012559571205948304
KRE -> Beta:  1.9819645983765466
KRE -> Tau:  0.013884140086908893

KBWB -> Expected Daily Return in %:  0.12738072558728253
KBWB -> Alpha:  -0.0005465133548920583
KBWB -> Beta:  1.6333946386633564
KBWB -> Tau:  0.007547225427195218



In [61]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [62]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [63]:
simm_weights_with_ss_dict = {
  "XLF": simm_weights_with_ss[0],
  "VFH": simm_weights_with_ss[1],
  "IYF": simm_weights_with_ss[2],
  "KRE": simm_weights_with_ss[3],
  "KBWB": simm_weights_with_ss[4]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

XLF -> SIMM Weights with Short Selling in %:  271.94240392194115
VFH -> SIMM Weights with Short Selling in %:  -76.88880222720135
IYF -> SIMM Weights with Short Selling in %:  -42.51522605081295
KRE -> SIMM Weights with Short Selling in %:  -16.589555197343955
KBWB -> SIMM Weights with Short Selling in %:  -35.94882044658291


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [64]:
simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_with_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)

Portfolio Risk with Short Selling in %:  0.5878843779765633


In [65]:
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  1.755038628200499


In [66]:
# find the sharpe ratio
simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

SIMM Sharpe Ratio with Short Selling:  0.22432332016545753


In [67]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [68]:
simm_weights_no_ss_dict = {
  "XLF": simm_weights_no_ss[0],
  "VFH": simm_weights_no_ss[1],
  "IYF": simm_weights_no_ss[2],
  "KRE": simm_weights_no_ss[3],
  "KBWB": simm_weights_no_ss[4]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

XLF -> SIMM Weights without Short Selling in %:  77.12889015060024
VFH -> SIMM Weights without Short Selling in %:  12.946577990349875
IYF -> SIMM Weights without Short Selling in %:  8.844714500555307
KRE -> SIMM Weights without Short Selling in %:  0.28498403629605945
KBWB -> SIMM Weights without Short Selling in %:  0.7948333221985013


In [69]:
# we will now work to find the expected return and risk of the portfolio
simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_no_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)

Portfolio Risk without Short Selling in %:  0.8282001522286906


In [70]:
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  1.9091534133527333


In [71]:
# find the sharpe ratio
simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

SIMM Sharpe Ratio without Short Selling:  0.2077557653005163


On running SIMM strategy, we observe higher mean portfolio return when short sell is not allowed.